In [208]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, display

# Xarray

Xarray, datos etiquetados y multidimensionalidad.
- Domain agnostic
- Dimensions
- Metadata
- Open source

<img src="images/xarray.png" width="1024" height="512">


## Data structures

<img src="images/data_structures.png"  width="1280" height="300">

**Data array** guarda variables individuales multi-dimensionales, sus coordenadas y metadata.

**Data set** guarda múltiples variables (que potencialmente comparten las mismas coordenadas).


## DataArray 📗

array de datos multidimensional con: 
- **Nombre** del array
- **Dimensiones** label que le doy a mis DataArrays
- **Coordenadas** valores asociados a mis dimensiones
- **Atributos** diccionario donde guardar metadata

<img src="images/tag.png">

##### Ejemplo

In [230]:
da = xr.DataArray(
    data = np.array([[["hola"],["adios"]],[["perro"],["gato"]],[["día"],["noche"]]]), 
    dims = ["mi_lista","pares","palabra"],
    coords = {
        "mi_lista": ["saludos", "animales", "cielo"],
        "pares": list(range(1, 3)),
        "palabra": [0],
    },
    name = "ejemplo",
    attrs={"fuente" : "de los deseos"}
)

|"mi_lista" | "saludos"         |  "animales"          |  "cielo"            |
|--------|----------------------|----------------------|---------------------|
|        | [["hola"],["adios"]] | [["perro"],["gato"]] | [["día"],["noche"]] |

|"pares" | 1                    |  2                   |
|--------|----------------------|----------------------|
|        | ["hola"]             | ["adios"]            |


|"palabra" | 0                    |
|--------|----------------------|
|        | "hola"             |

example

<img src="images/example_DataArray.png" width="1486">

In [211]:
example2_data = np.array([[0.63865377,0.41891651,0.26028003],[0.50072503,0.8474092,0.24947729]])

##### Otro ejemplo

In [213]:
example2 = xr.DataArray(
    data = example2_data, 
    dims = ["x","y"],
    coords = {
        "x": [-10,10],
        "y": [-20,0,20],
    },
    name = "ejemplo 2",
)

| x\y  | -20 | 0 | 20 |
|-----|-------------|-----|-----|
| -10  | 0.63865377 | 0.41891651 | 0.26028003 |
|  10  | 0.50072503 | 0.8474092  | 0.24947729 |

## Dataset 📚

Especie de diccionario que contiene labeled DataArrays y metadata.      
(Esos DataArrays están _alineados_ (mismo tamaño)).      
Diseñado para representar la data del modelo netCDF.
- **Dimensiones** diccionario con [nombre de dimension]:[tamaño fijo]
- **Coordenadas** valores de esas dimensiones (la intención es ponerle label a las _data_vars_)
- **Data variables** diccionario de variables (_data_vars_)
- **Atributos** metadata extra

### Convención
*Coordenadas* ➡️ cantidades constantes/fijas/independientes (no cambia)          
*Data variables* ➡️ cantidades variantes/dependientes (¡está siendo medido!) ➡️ lo quiero analizar 🔍

##### Ejemplo

In [214]:
ds = xr.Dataset(
    data_vars={
        "temperatura": (["lat", "lon"], [[15, 16], [17, 18]]), # DataArrays donde dim=["lat", "lon"]
        "humedad":     (["lat", "lon"], [[60, 65], [70, 75]])
    },
    coords={
        "lat": [10, 20],
        "lon": [100, 110]
        # podría tener por ejemplo "tiempo"
    }
)

In [215]:
# ahora solo vista text para ahorrar espacio
xr.set_options(display_style="text")

## Indexing con Xarray


Una vez tengamos nuestro data array queremos "aprovechar" nuestras coordenas, y por algun motivo no perder la posibilidad de ser infelices:         
Para ello podemos aprovecharlos y seleccionar a traves indices (porque wtf?) o a traves de nombres dimension:

In [216]:
example = xr.DataArray(
    np.arange(0, 60).reshape(3, 4, 5),
    coords={
        "dim1": ["A", "B", "C"],
        "dim2": ["a", "b", "c", "d"],
        "dim3": np.arange(0, 5)
    },
    dims=("dim1", "dim2", "dim3")
)

In [217]:
example[2,1,2]

<xarray.DataArray ()>
array(47)
Coordinates:
    dim1     <U1 'C'
    dim2     <U1 'b'
    dim3     int64 2

In [218]:
example.loc["C","b",2]

<xarray.DataArray ()>
array(47)
Coordinates:
    dim1     <U1 'C'
    dim2     <U1 'b'
    dim3     int64 2

In [219]:
example.sel(dim1 = slice("A","C"))

<xarray.DataArray (dim1: 3, dim2: 4, dim3: 5)>
array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]],

       [[20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39]],

       [[40, 41, 42, 43, 44],
        [45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54],
        [55, 56, 57, 58, 59]]])
Coordinates:
  * dim1     (dim1) <U1 'A' 'B' 'C'
  * dim2     (dim2) <U1 'a' 'b' 'c' 'd'
  * dim3     (dim3) int64 0 1 2 3 4

In [220]:
example.isel(dim1 = 0)

<xarray.DataArray (dim2: 4, dim3: 5)>
array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19]])
Coordinates:
    dim1     <U1 'A'
  * dim2     (dim2) <U1 'a' 'b' 'c' 'd'
  * dim3     (dim3) int64 0 1 2 3 4

In [221]:
example.drop_sel(dim3 = [2,3])

<xarray.DataArray (dim1: 3, dim2: 4, dim3: 3)>
array([[[ 0,  1,  4],
        [ 5,  6,  9],
        [10, 11, 14],
        [15, 16, 19]],

       [[20, 21, 24],
        [25, 26, 29],
        [30, 31, 34],
        [35, 36, 39]],

       [[40, 41, 44],
        [45, 46, 49],
        [50, 51, 54],
        [55, 56, 59]]])
Coordinates:
  * dim1     (dim1) <U1 'A' 'B' 'C'
  * dim2     (dim2) <U1 'a' 'b' 'c' 'd'
  * dim3     (dim3) int64 0 1 4

In [222]:
example.drop_sel(dim3 = [2,3])

<xarray.DataArray (dim1: 3, dim2: 4, dim3: 3)>
array([[[ 0,  1,  4],
        [ 5,  6,  9],
        [10, 11, 14],
        [15, 16, 19]],

       [[20, 21, 24],
        [25, 26, 29],
        [30, 31, 34],
        [35, 36, 39]],

       [[40, 41, 44],
        [45, 46, 49],
        [50, 51, 54],
        [55, 56, 59]]])
Coordinates:
  * dim1     (dim1) <U1 'A' 'B' 'C'
  * dim2     (dim2) <U1 'a' 'b' 'c' 'd'
  * dim3     (dim3) int64 0 1 4

In [223]:
example2 = example.assign_coords(dim1 = [97.8,98.2,344.23123])
example2

<xarray.DataArray (dim1: 3, dim2: 4, dim3: 5)>
array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]],

       [[20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39]],

       [[40, 41, 42, 43, 44],
        [45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54],
        [55, 56, 57, 58, 59]]])
Coordinates:
  * dim1     (dim1) float64 97.8 98.2 344.2
  * dim2     (dim2) <U1 'a' 'b' 'c' 'd'
  * dim3     (dim3) int64 0 1 2 3 4

In [224]:
example2.sel(dim1 = 97)

KeyError: "not all values found in index 'dim1'. Try setting the `method` keyword argument (example: method='nearest')."

## DateTime Indexing:

Podes tener una dimension que sus coordenas sean fechas, entonces logicamente deberia andar bien el seleccionar una fecha puntual.. anda bien :D, pero no solo eso sino que dado que esta guardado en formato fecha podes hacer cosas como:
| Objetivo                                                            | Código `xarray`                                                             | Tipo             |
| ------------------------------------------------------------------- | --------------------------------------------------------------------------- | ---------------- |
| Fecha exacta (día y hora)                                           | `ds.sel(time='2013-01-01 06:00')`                                           | Exacta           |
| Día completo (todas las horas)                                      | `ds.sel(time='2013-01-01')`                                                 | Exacta           |
| Mes completo (todas las fechas de ese mes y año)                    | `ds.sel(time='2014-05')`                                                    | Exacta           |
| Año completo                                                        | `ds.sel(time='2013')`                                                       | Exacta           |
| Slice entre dos fechas                                              | `ds.sel(time=slice('2013-01-01', '2013-01-10'))`                            | Slice            |
| Slice entre dos fechas con hora                                     | `ds.sel(time=slice('2013-01-01 03:00', '2013-01-02 12:00'))`                | Slice            |
| Slice de mes a mes                                                  | `ds.sel(time=slice('2013-01', '2013-03'))`                                  | Slice            |
| Slice de año a año                                                  | `ds.sel(time=slice('2012', '2014'))`                                        | Slice            |
| Todos los valores de un **mes** sin importar año                    | `ds.sel(time=ds.time.dt.month == 7)`                                        | Condicional      |
| Todos los valores de un **día del mes** sin importar mes/año        | `ds.sel(time=ds.time.dt.day == 15)`                                         | Condicional      |
| Todos los valores de un **año específico**                          | `ds.sel(time=ds.time.dt.year == 2014)`                                      | Condicional      |
| Todos los valores de una **hora específica**                        | `ds.sel(time=ds.time.dt.hour == 6)`                                         | Condicional      |
| Todos los valores de un **día de la semana (0=lunes)**              | `ds.sel(time=ds.time.dt.dayofweek == 0)`                                    | Condicional      |
| Todos los valores de una **fecha específica combinada (mes + día)** | `ds.sel(time=(ds.time.dt.month == 12) & (ds.time.dt.day == 24))`            | Combinada        |
| Estación: Verano (dic-ene-feb)                                      | `ds.sel(time=ds.time.dt.month.isin([12, 1, 2]))`                            | Estacional       |
| Estación: Otoño (mar-abr-may)                                       | `ds.sel(time=ds.time.dt.month.isin([3, 4, 5]))`                             | Estacional       |
| Estación: Invierno (jun-jul-ago)                                    | `ds.sel(time=ds.time.dt.month.isin([6, 7, 8]))`                             | Estacional       |
| Estación: Primavera (sep-oct-nov)                                   | `ds.sel(time=ds.time.dt.month.isin([9, 10, 11]))`                           | Estacional       |
| Estación combinada con año (ej: invierno 2013)                      | `ds.sel(time=(ds.time.dt.year == 2013) & ds.time.dt.month.isin([6, 7, 8]))` | Estacional + Año |
| Días impares                                                        | `ds.sel(time=ds.time.dt.day % 2 == 1)`                                      | Personalizada    |
| Último día del mes                                                  | `ds.sel(time=ds.time.dt.is_month_end)`                                      | Especial         |
| Primer día del mes                                                  | `ds.sel(time=ds.time.dt.is_month_start)`                                    | Especial         |


## COMPUTATION
Probablemente querramos hacer cosas mas interesantes que preguntar por datos en nuestro xarray, asi que veamos que podemos hacer :D
### BASIC
Podemos aplicar funciones y hacer cualquier reduccion estandar de numpy en las coordenadas (mean, min, max, sum, etc.)

In [225]:
example = xr.DataArray(
    np.zeros((3,4)),
    coords = {
        "x" : np.arange(0,3),
        "y" : np.arange(0,4)
    },
    dims = ("x","y")
)
example

<xarray.DataArray (x: 3, y: 4)>
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])
Coordinates:
  * x        (x) int64 0 1 2
  * y        (y) int64 0 1 2 3

In [226]:
def sumar(a,b):
    return a+b

xr.apply_ufunc(sumar,example,2)

<xarray.DataArray (x: 3, y: 4)>
array([[2., 2., 2., 2.],
       [2., 2., 2., 2.],
       [2., 2., 2., 2.]])
Coordinates:
  * x        (x) int64 0 1 2
  * y        (y) int64 0 1 2 3

In [227]:
from math import *
(example + 2) ** 2

<xarray.DataArray (x: 3, y: 4)>
array([[4., 4., 4., 4.],
       [4., 4., 4., 4.],
       [4., 4., 4., 4.]])
Coordinates:
  * x        (x) int64 0 1 2
  * y        (y) int64 0 1 2 3

In [228]:
arr = np.array([
    [0, 1, 2, 3],
    [8, 9, 10, 11],
    [0, 5, 6, 7]
])
example.data = arr
example

<xarray.DataArray (x: 3, y: 4)>
array([[ 0,  1,  2,  3],
       [ 8,  9, 10, 11],
       [ 0,  5,  6,  7]])
Coordinates:
  * x        (x) int64 0 1 2
  * y        (y) int64 0 1 2 3

In [229]:
example.mean(dim = "x")

<xarray.DataArray (y: 4)>
array([2.66666667, 5.        , 6.        , 7.        ])
Coordinates:
  * y        (y) int64 0 1 2 3